# BioE 131 LAB 4 | Connor Tou

As shown in the below table, 4 enzymes were chosen (ordered) from 3 pathways by looking at the graphical pathways on KEGG. The EC (Enzyme Comission) Number is shown. 

| PATHWAY           | ENZYME 1 | ENZYME 2 | ENZYME 3 | ENZYME 4 |
|-------------------|----------|----------|----------|----------|
| Glycolysis        | 5.4.2.2  | 5.3.1.9  | 4.1.1.49 | 6.2.1.13 |
| TCA               | 1.1.5.4  | 4.1.1.31 | 4.1.1.49 | 4.1.1.32 |
| Pentose Phosphate | 2.7.1.13 | 4.1.2.43 | 5.3.1.6  | 5.4.2.7  |

### DATABASE SCHEMA

#### GENES TABLE

| ID | NAME | ORGNANISM |   |   |
|----|------|-----------|---|---|
|    |      |           |   |   |
|    |      |           |   |   |
|    |      |           |   |   |

36 genes will be stored in this table: 3 pathways, 4 genes, 3 organisms (homo sapiens, drosophilia melanogaster, and escherichia coli K-12 MG165)

#### PATHWAYS TABLE

| ID | NAME | DESCRIPTION |
|----|------|-------------|
|    |      |             |
|    |      |             |

Information for the 3 pathways are stored here. Note that Descriptions were taken directly from KEGG. 

#### ENZYME TABLE

| ID | NAME | DESCRIPTION | Enzyme Commission # |
|----|------|-------------|---------------------|
|    |      |             |                     |
|    |      |             |                     |

The information for 12 enzymes will be stored here. EC # is used. 

#### ASSOCIATIVE TABLE 1: PATHWAY_ENZYME TABLE

| ID | PATHWAY | ENZYME | ORDER |
|----|---------|--------|-------|
|    |         |        |       |
|    |         |        |       |

The relationship between pathways and enzymes (and the enzyme's order in a given pathway) are stored here. **Note that some enzymes belong to many pathways - this is a one to many relationship.**

#### ASSOCIATIVE TABLE 2: ENZYME_GENE TABLE

| ID | ENZYME COMMISSION # | ORGANISM | GENE |
|----|---------------------|----------|------|
|    |                     |          |      |
|    |                     |          |      |

The relationship between enzymes and genes are stored here. **Note that a single gene can be part of many enzymes (through alternativ and a single enzyme contains many coding genes. This is a many to many relationship.**

In [2]:
import sqlite3
import pandas as pd
from Bio import Entrez
from Bio import SeqIO
import sys

Entrez.email = "connortou@berkeley.edu"
conn = sqlite3.connect('Lab4.db')
c = conn.cursor()

# If these SQL tables already exist, delete them
try:
    print("DROPPING TABLES...BEEP..BOOP...MAKING NEW ONES...BEEP..BOOP..BEEP..BOOP")
    c.execute("DROP TABLE genes")
    c.execute("DROP TABLE pathways")
    c.execute("DROP TABLE enzymes")
    c.execute("DROP TABLE pathway_enzyme")
    c.execute("DROP TABLE enzyme_gene")
except:
    print("MAKING TABLES...BEEP...BOOP")
    
    
# Create these SQL tables
c.execute("""CREATE TABLE genes (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                name TEXT,
                                organism TEXT,
                                sequence TEXT);""")


c.execute("""CREATE TABLE pathways (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    name TEXT,
                                    description TEXT);""")

c.execute("""CREATE TABLE enzymes (ec TEXT PRIMARY KEY,
                                    name TEXT,
                                    description TEXT);""")

c.execute("""CREATE TABLE pathway_enzyme (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                            pathway TEXT,
                                            enzyme TEXT,
                                            pathway_order INTEGER);""")


c.execute("""CREATE TABLE enzyme_gene (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                            ec TEXT,
                                            organism TEXT,
                                            gene TEXT);""")

# Considered organisms, pathways and enzymes. Enzymes are given in order of occurence in the pathway.
organism = ['homo sapiens', 'escherichia coli', 'drosophila']

path = {'Glycolysis': ['5.4.2.2','5.3.1.9', '4.1.1.49','6.2.1.13'], 
      'TCA cycle':['1.1.5.4', '4.1.1.31','4.1.1.49','4.1.1.32'],
      'Pentose Phosphate':['2.7.1.13', '4.1.2.43', '5.3.1.6', '5.4.2.7']}

# Enter Descriptions (Pulled from KEGG)
c.execute("""INSERT INTO pathways (name,
                                    description)
                            VALUES
                                    ('Glycolysis',
                                    'Glycolysis is the process of converting glucose into pyruvate and generating small 
                                    amounts of ATP (energy) and NADH (reducing power). It is a central pathway that 
                                    produces important precursor metabolites: six-carbon compounds of glucose-6P and 
                                    fructose-6P and three-carbon compounds of glycerone-P, glyceraldehyde-3P, 
                                    glycerate-3P, phosphoenolpyruvate, and pyruvate. Acetyl-CoA, another important 
                                    precursor metabolite, is produced by oxidative decarboxylation of pyruvate. When the
                                    enzyme genes of this pathway are examined in completely sequenced genomes, the 
                                    reaction steps of three-carbon compounds from glycerone-P to pyruvate form a 
                                    conserved core module, which is found in almost all organisms and which sometimes
                                    contains operon structures in bacterial genomes. Gluconeogenesis is a synthesis 
                                    pathway of glucose from noncarbohydrate precursors. It is essentially a reversal of 
                                    glycolysis with minor variations of alternative paths.')
                                    ;""")

c.execute("""INSERT INTO pathways (name,
                                    description)
                            VALUES
                                    ('TCA cycle',
                                    'The citrate cycle (TCA cycle, Krebs cycle) is an important aerobic pathway for the 
                                    final steps of the oxidation of carbohydrates and fatty acids. The cycle starts with 
                                    acetyl-CoA, the activated form of acetate, derived from glycolysis and pyruvate 
                                    oxidation for carbohydrates and from beta oxidation of fatty acids. The two-carbon 
                                    acetyl group in acetyl-CoA is transferred to the four-carbon compound of oxaloacetate 
                                    to form the six-carbon compound of citrate. In a series of reactions two carbons in 
                                    citrate are oxidized to CO2 and the reaction pathway supplies NADH for use in the 
                                    oxidative phosphorylation and other metabolic processes. The pathway also supplies 
                                    important precursor metabolites including 2-oxoglutarate. At the end of the cycle the 
                                    remaining four-carbon part is transformed back to oxaloacetate. According to the 
                                    genome sequence data, many organisms seem to lack genes for the full cycle, but 
                                    contain genes for specific segments.')
                                    ;""")

c.execute("""INSERT INTO pathways (name,
                                    description)
                            VALUES
                                    ('Pentose Phosphate',
                                    'The pentose phosphate pathway is a process of glucose turnover that produces NADPH 
                                    as reducing equivalents and pentoses as essential parts of nucleotides. There are two 
                                    different phases in the pathway. One is irreversible oxidative phase in which 
                                    glucose-6P is converted to ribulose-5P by oxidative decarboxylation, and NADPH is 
                                    generated. The other is reversible non-oxidative phase in which phosphorylated sugars 
                                    are interconverted to generate xylulose-5P, ribulose-5P, and ribose-5P. 
                                    Phosphoribosyl pyrophosphate (PRPP) formed from ribose-5P is an activated compound 
                                    used in the biosynthesis of histidine and purine/pyrimidine nucleotides. This pathway 
                                    map also shows the Entner-Doudoroff pathway where 6-P-gluconate is dehydrated and then
                                    cleaved into pyruvate and glyceraldehyde-3P.')
                                    ;""")

# Extract data from Entrez automatically to fill other tables
for (pathway, enzyme) in path.items():
    # number of pathway enzyme
    num_enzyme = 1
    for e in enzyme:
        # Create associated pathway linking pathways and enzymes
        c.execute("""INSERT INTO pathway_enzyme (pathway,
                                                enzyme,
                                                pathway_order)
                                        VALUES
                                                (?,
                                                ?,
                                                ?);""" ,
                                                (pathway,e,num_enzyme))
        num_enzyme +=1
             
       
        
        # orgnanism sequence
        for o in organism:
            # The current search is printed to give the user an indication of progress
            print('Pathway: ' + pathway + '; EC: ' + e + '; Organism: ' + o)

             # Search for info using Entrez
            handle = Entrez.esearch(db = 'protein',
                                    term = 'EC: ' + e + ' AND ' + o,
                                    sort = 'relevance',
                                    idtype = 'acc',
                                    retmax = 1)
            
            for i in Entrez.read(handle)['IdList']:
                handle = Entrez.efetch(db = 'protein', id = i, rettype = 'gb', retmode = 'text')
                record = SeqIO.read(handle, "genbank")
                handle.close()
                c.execute("""INSERT OR REPLACE INTO enzymes (name,
                                                description,
                                                ec)
                                        VALUES
                                                (?,
                                                ?,
                                                ?);""" ,
                                                (record.name,record.description,e))
                
            # The corresponding nucleotide sequence information is searched for using Entrez
            # The sequence is found using the gene database, the rest of the information is found using the nucleotide one
            handle = Entrez.esearch(db = 'gene',
                                    term = e + ' ' + o,
                                    sort = 'relevance',
                                    idtype = 'acc',
                                    retmax = 1)
            
            
            for i in Entrez.read(handle)['IdList']:
                handle = Entrez.efetch(db = 'nucleotide', id = i, rettype = 'gb', retmode = 'text')
                record = SeqIO.read(handle, "genbank")
                handle.close()

                c.execute("""INSERT INTO genes (name,
                                                organism,
                                                sequence)
                                        VALUES
                                                (?,
                                                ?,
                                                ?);""" ,
                                                (record.id, o, str(record.seq)))
        
                c.execute("""INSERT INTO enzyme_gene (ec,
                                                    organism,
                                                    gene)
                                            VALUES
                                                    (?,
                                                    ?,
                                                    ?);""" ,
                                                    (e,o,record.id))
            

# Update tables
conn.commit()

# Print tables
print("\nPathways table:")
print (pd.read_sql_query("SELECT * FROM pathways", conn))
print("\nGenes table:")
print (pd.read_sql_query("SELECT * FROM genes", conn))
print("\nEnzymes table:")
print (pd.read_sql_query("SELECT * FROM enzymes", conn))
print("\nPathway enzyme associated table:")
print (pd.read_sql_query("SELECT * FROM pathway_enzyme", conn))
print("\nEnzyme genes associated table:")
print (pd.read_sql_query("SELECT * FROM enzyme_gene", conn))

DROPPING TABLES...BEEP, BOOP,BEEP, BOOP...MAKING NEW ONES...BEEP, BOOP, BEEP, BOOP
Pathway: Glycolysis; EC: 5.4.2.2; Organism: homo sapiens
Pathway: Glycolysis; EC: 5.4.2.2; Organism: escherichia coli
Pathway: Glycolysis; EC: 5.4.2.2; Organism: drosophila
Pathway: Glycolysis; EC: 5.3.1.9; Organism: homo sapiens
Pathway: Glycolysis; EC: 5.3.1.9; Organism: escherichia coli
Pathway: Glycolysis; EC: 5.3.1.9; Organism: drosophila
Pathway: Glycolysis; EC: 4.1.1.49; Organism: homo sapiens
Pathway: Glycolysis; EC: 4.1.1.49; Organism: escherichia coli
Pathway: Glycolysis; EC: 4.1.1.49; Organism: drosophila
Pathway: Glycolysis; EC: 6.2.1.13; Organism: homo sapiens
Pathway: Glycolysis; EC: 6.2.1.13; Organism: escherichia coli
Pathway: Glycolysis; EC: 6.2.1.13; Organism: drosophila
Pathway: TCA cycle; EC: 1.1.5.4; Organism: homo sapiens
Pathway: TCA cycle; EC: 1.1.5.4; Organism: escherichia coli
Pathway: TCA cycle; EC: 1.1.5.4; Organism: drosophila
Pathway: TCA cycle; EC: 4.1.1.31; Organism: homo 

#### Try UCSC, Entrez, KEGG, and Reactome. Notice how each database presents the information in a different way. 

1.) Do you have any preference? 
* I like the interactive nature of Reactome; however, I find it a bit clunkier and harder to use/navigate. For utility purposes, I prefer KEGG or UCSC.

2.) Which seems to be the most complete? 
* KEGG and UCSC both have extensive data and seems the most complete. 

# END OF LAB 4